In [25]:
import pandas as pd 
import numpy as  np 
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error
import pickle

In [26]:
test = pd.read_csv("ApartmentRentPrediction.csv")

In [27]:
columns =["amenities","bedrooms","bathrooms","square_feet","latitude","longitude",
          "cityname","state","price_display"]
test = test[columns]

In [28]:
with open("mode_values.pickle", "rb") as f:
    loaded_mode_mean_values = pickle.load(f)

pets_allowed_mode= loaded_mode_mean_values["pets_allowed_mode"]
cityname_mode= loaded_mode_mean_values["cityname_mode"]
state_mode = loaded_mode_mean_values["state_mode"]
bedroom_mean = loaded_mode_mean_values["bedrooms_mean"]
bathroom_mean = loaded_mode_mean_values["bathrooms_mean"]
long_mean = loaded_mode_mean_values["longitude_mean"]
latit_mean = loaded_mode_mean_values["latitude_mean"]


test["amenities"].fillna("Parking", inplace=True)
test["cityname"].fillna(cityname_mode, inplace=True)
test["state"].fillna(state_mode, inplace=True)
test["bedrooms"].fillna(bedroom_mean, inplace=True)
test["bathrooms"].fillna(bathroom_mean, inplace=True)
test["longitude"].fillna(long_mean, inplace=True)
test["latitude"].fillna(latit_mean, inplace=True)

In [29]:
with open('top_15_words.pkl', 'rb') as f:
    top_15_words = pickle.load(f)
    
for i, row in test.iterrows():
    text = row['amenities']
    found = False  # Initialize found flag for each row
    a = 0
    for word in top_15_words:
        if word in text:
            found = True
            text = text.replace(word, str(top_15_words.index(word) + 1))
            a = a+1
    
    text  = str(a)
    if not found:
        text = str(0)
        
    test.at[i, 'amenities'] = text
test["amenities"]=test["amenities"].astype(int)

In [30]:
def in_area(row):
    if (-123 <= row['longitude'] <= -114) and (30 <= row['latitude'] <= 38):
        return 1
    else:
        return 0
    
test['in_area'] = test.apply(in_area, axis=1)

In [31]:
import pickle

with open('city_offer_map.pkl', 'rb') as f:
    city_offer_map = pickle.load(f)

test['city_offer_what'] = test['cityname'].map(city_offer_map)

In [32]:
with open('label_encoders_reg.pickle', 'rb') as f:
    loaded_label_encoders = pickle.load(f)
    
columns = ['cityname', 'state','city_offer_what']

for column in columns:
    encoder = loaded_label_encoders[column]
#    print(encoder.classes_)
    test[column + '_encoded'] = encoder.transform(test[column])

    

test["city_encoded"]= test["cityname_encoded"]
test.drop(columns = "cityname_encoded",axis=1,inplace=True)
# test[loaded_label_encoders[columns[0]]] = encoder.transform(test[columns[0]])

# test[loaded_label_encoders[columns[1]]] = encoder.transform(test[columns[1]])

# test[loaded_label_encoders[columns[2]]] = encoder.transform(test[columns[1]])

In [33]:
test.drop(columns=["cityname","state","city_offer_what"],axis=1,inplace=True)

In [34]:

test['price_display'] = test['price_display'].str.replace(r'[^\d.]', '', regex=True).astype(float)

In [35]:
column_order = ['amenities', 'bathrooms', 'bedrooms', 'price_display', 
                'square_feet', 'latitude', 'longitude', 'in_area', 
                'city_encoded', 'state_encoded', 'city_offer_what_encoded']

# Reordering columns
test = test.reindex(columns=column_order)

# Displaying DataFrame info
#test.info()

In [36]:
test["price_display"].describe()

count     9000.000000
mean      1487.286222
std       1088.561190
min        200.000000
25%        950.000000
50%       1275.000000
75%       1695.000000
max      52500.000000
Name: price_display, dtype: float64

In [37]:
with open('random_forest_reg_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
    
X=test.drop(columns="price_display",axis=1)
y=test["price_display"]


predictions = loaded_model.predict(X)

mean_squared_error(y,predictions)


449844.53041715076

In [38]:
r2_score(y,predictions)

0.6203311135621741